In [12]:
'''
    File name: convert-trace-axograph.ipynb
    Author: Mustafa S. Hamada, PhD
    Date created: 2021-04-21
    Date last modified: 2021-05-30
    Python Version: 3.9
'''

from allensdk.core.cell_types_cache import CellTypesCache

# Instantiate the CellTypesCache instance.  The manifest_file argument
# tells it where to store the manifest, which is a JSON file that tracks
# file paths.  If you supply a relative path it will go into your
# current working directory
ctc = CellTypesCache()

# this saves the NWB file to 'cell_types/specimen_464212183/ephys.nwb'
cell_specimen_id = 571642127  # replace with relevant specimen_id
data_set = ctc.get_ephys_data(cell_specimen_id)

In [13]:
# %matplotlib inline  # allows output graph to be printed inline if running in Jupyter Lab
# import init_CellTypesCache # when splitting the blocks into individual files
import axographio
import numpy as np

# Input manually the designated trace numbers from the neuron 'Browse Electrophysiological Data' section. The traces
# below are from a human neuron ID = 571471295 (https://celltypes.brain-map.org/experiment/electrophysiology/571471295)
neuralTraces = [21, 22, 23, 25, 19, 67, 65]

for trace in neuralTraces:
        fileName = 'cell_id' + str(cell_specimen_id) + '--trace--' + str(trace) + '.axgx'
        
        trace_data = data_set.get_sweep(trace)
        index_range = trace_data["index_range"]

        inputCurrent = trace_data["stimulus"][0:index_range[1] + 1]  # in Ampere
        voltageResponse = trace_data["response"][0:index_range[1] + 1]  # in Voltage

        inputCurrent *= 1e12  # to pico-Ampere (pA)
        voltageResponse *= 1e3  # to milli-Voltage (mV)

        sampling_rate = trace_data["sampling_rate"]  # in Hz

        time = np.arange(0, len(voltageResponse)) * (1.0 / sampling_rate)  # in seconds
        
        np.prod(inputCurrent.shape)

        _trace = axographio.file_contents(
            ['time (s)', 'input current (pA)', 'voltage reponse (mV)'],
            [time, inputCurrent, voltageResponse])

        _trace.write(fileName)

print(sampling_rate / 1000, "kHz")  # outputs sampling rate frequency